<a href="https://colab.research.google.com/github/SahasKoka/CyberbullyingClassifactionProject/blob/main/BERT_ModelTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install transformers datasets
!pip install accelerate -U

# Check GPU availability
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU is not available")

# Load the dataset
from datasets import load_dataset, load_metric
import numpy as np
dataset = load_dataset("sahask8/cb_train_set", split="train")

def is_valid_text(text):
    """Checks if the given text is a valid string and not empty."""
    return isinstance(text, str) and text.strip() and len(text.strip()) > 0 and  len(text.strip()) < 128

def filter_invalid_text(example):
    """Returns True if the 'Text' column contains valid text, False otherwise."""
    return is_valid_text(example["Text"]) and example["oh_label"] is not None

dataset = dataset.filter(filter_invalid_text)

# Load model and tokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)

# Tokenize the dataset
def tokenize_function(example):

    tokenized  = tokenizer(str(example["Text"]), truncation=True, padding="max_length", max_length=128)
    tokenized["labels"] = int(example["oh_label"])
    return tokenized

tokenized_datasets = dataset.map(tokenize_function, batched=False)

# Load the metric calculation function
metric = load_metric("precision")

# Define a function to compute metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

#trainer.save_model("./cb_bert_fine_tuned_model")

import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from datasets import load_dataset

# Load the saved model and tokenizer
#model_path = "./cb_bert_fine_tuned_model"
#tokenizer = AutoTokenizer.from_pretrained(model_path)
#model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Load the test dataset
test_dataset = load_dataset("sahask8/cb_test_set")  # Assuming "validation" is your test split

def predict_sentiment(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device) # Move inputs to GPU

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Get predicted class
    predicted_class = torch.argmax(logits, dim=1).item()
    return predicted_class


# Get predictions for the test set
true_labels = []
predicted_labels = []
#print(test_dataset)
for example in test_dataset['test']:
    true_labels.append(int(example["oh_label"]))
    predicted_labels.append(predict_sentiment(example["Text"]))

# Calculate metrics
accuracy = accuracy_score(true_labels, predicted_labels)
precision = precision_score(true_labels, predicted_labels)
recall = recall_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/16514 [00:00<?, ? examples/s]

Filter:   0%|          | 0/16514 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision
1,0.372900,0.245496,0.795240
2,0.288700,0.158004,0.932779


Epoch,Training Loss,Validation Loss,Precision
1,0.372900,0.245496,0.795240
2,0.288700,0.158004,0.932779
3,0.194900,0.103025,0.953488


Generating test split:   0%|          | 0/337 [00:00<?, ? examples/s]

Accuracy: 0.8516320474777448
Precision: 0.7878787878787878
Recall: 0.7289719626168224
F1-score: 0.7572815533980582
